In [1]:
import pandas as pd 
import numpy as np
from soma import aims
import os

In [14]:
df = pd.read_csv('../../SourceData/left_palette.csv', sep=',')

In [16]:
paleta = ''
for i, row in df.iterrows():
    paleta = paleta + '{};{};'.format(row['label'],row.R)
paleta = paleta[:-1] + '#'

for i, row in df.iterrows():
    paleta = paleta + '{};{};'.format(row['label'],row.G)
paleta = paleta[:-1] + '#'

for i, row in df.iterrows():
    paleta = paleta + '{};{};'.format(row['label'],row.B)

paleta = paleta[:-1] + '#0.5;1'

In [18]:
with open('../../SourceData/Neonate_atlas_paleta_left.csv', 'w+') as fh:
    fh.write(paleta)


In [5]:
df = pd.read_csv('../../SourceData/paleta_r.csv', sep=',')

paleta = ''
for i, row in df.iterrows():
    paleta = paleta + '{};{};'.format(row['Label'],row.R)
paleta = paleta[:-1] + '#'

for i, row in df.iterrows():
    paleta = paleta + '{};{};'.format(row['Label'],row.G)
paleta = paleta[:-1] + '#'

for i, row in df.iterrows():
    paleta = paleta + '{};{};'.format(row['Label'],row.B)

paleta = paleta[:-1] + '#0.5;1'

with open('../../SourceData/Neonate_atlas_paleta_right.csv', 'w+') as fh:
    fh.write(paleta)


In [4]:
df = pd.read_csv('../../SourceData/paleta_combined.csv', sep=',', names=['Label', 'R', 'G', 'B'])

paleta = ''
for i, row in df.iterrows():
    paleta = paleta + '{};{};'.format(row['Label'],row.R)
paleta = paleta[:-1] + '#'

for i, row in df.iterrows():
    paleta = paleta + '{};{};'.format(row['Label'],row.G)
paleta = paleta[:-1] + '#'

for i, row in df.iterrows():
    paleta = paleta + '{};{};'.format(row['Label'],row.B)

paleta = paleta[:-1] + '#0.5;1'

with open('../../SourceData/Neonate_atlas_paleta_combined.csv', 'w+') as fh:
    fh.write(paleta)


In [2]:
## paleta vobec nefunguje lebo anatomist je pravdepodobne ten najhlupejsi software na visualizaciu, 
## skusim pre-labelovat jednu hemi na druhu aby som pouzila jednotnu schemu? 
sub = pd.read_csv('../../Scripts/data_processing/subj_for_visualisation.csv', names=['subject_id', 'session_id', 'template'])

In [3]:
## attention - change from the previous where there was congulum gyral part, now only cingulate BUT hippocampus included
R_labels = [67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,103,105,107,109,111,113,117,121]
L_labels = [68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,118,122]

In [4]:
for i, row in sub.iterrows():
    
    iSegm = aims.read('../../DerivedData/subjects/sub-{}/ses-{}/sub-{}_ses-{}_NAt_cortex_shard_space.nii.gz'.format(
                    row.subject_id, row.session_id, row.subject_id, row.session_id))
    segm = np.array(iSegm)[:,:,:,0]
    
    for r, l in zip(R_labels, L_labels):
        segm[segm == r] = l
        
    new_im = aims.Volume(segm)
    new_im.header().update(iSegm.header())
        
    out = '../../Results/figures/example_subject_vis/sub-{}_ses-{}_NeonAtlas_shard_cortex_relabeled_to_match.nii.gz'.format(row.subject_id, row.session_id)
    aims.write(new_im, out)
        
    if os.path.isfile(out):
        print('{} {} relabelled'.format(row.subject_id, row.session_id))

CC00150AN02 54800 relabelled
CC00554XX10 160200 relabelled
CC00389XX19 119100 relabelled
CC00186AN14 60900 relabelled


In [13]:
## do the same thing for the surfaces 
# relabel right should be ok for left 

for i, row in sub.iterrows():
    
    h = 'R'
    hemi ='right'
    ## right
    mesh = aims.read('/neurospin/grip/external_databases/dHCP_CR_JD_2018/release3/dhcp_anat_pipeline/sub-{}/ses-{}/anat/sub-{}_ses-{}_T2w_{}white_bv_transformed.gii'.format(row.subject_id, row.session_id, row.subject_id, row.session_id,  h))
    vert, poly = np.array(mesh.vertex()) ,np.array(mesh.polygon())    
    iSegm = aims.read('../../DerivedData/subjects/sub-{}/ses-{}/sub-{}_ses-{}_neoAtlas_corrected_right.gii'.format(
                    row.subject_id, row.session_id, row.subject_id, row.session_id))
    segm = np.array(iSegm)[0]
    
    for r, l in zip(R_labels, L_labels):
        segm[segm == r] = l
        
    tex = aims.TimeTexture('FLOAT')  
    for i in range(mesh.size()):
        tex[i].assign(np.zeros((len(vert),), dtype=np.float32))
        t = np.asarray(tex[i])
        t[:] = segm[:]
        
    outf = '../../Results/figures/example_subject_vis/sub-{}_ses-{}_neoAtlas_corrected_{}_relabelled_to_match.gii'.format(row.subject_id, row.session_id, row.subject_id, row.session_id,   hemi)
    aims.write(tex, outf)   
    
    if os.path.isfile(outf):
        print(row.subject_id, '{} DONE'.format(hemi))
    
    h = 'L'
    hemi ='left'
    ## right
    mesh = aims.read('/neurospin/grip/external_databases/dHCP_CR_JD_2018/release3/dhcp_anat_pipeline/sub-{}/ses-{}/anat/sub-{}_ses-{}_T2w_{}white_bv_transformed.gii'.format(row.subject_id, row.session_id, row.subject_id, row.session_id,  h))
    vert, poly = np.array(mesh.vertex()) ,np.array(mesh.polygon())    
    iSegm = aims.read('../../DerivedData/subjects/sub-{}/ses-{}/sub-{}_ses-{}_neoAtlas_corrected_left.gii'.format(
                    row.subject_id, row.session_id, row.subject_id, row.session_id))
    segm = np.array(iSegm)[0]
    
    for r, l in zip(R_labels, L_labels):
        segm[segm == r] = l
        
    tex = aims.TimeTexture('FLOAT')  
    for i in range(mesh.size()):
        tex[i].assign(np.zeros((len(vert),), dtype=np.float32))
        t = np.asarray(tex[i])
        t[:] = segm[:]
        
    outf = '../../Results/figures/example_subject_vis/sub-{}_ses-{}_neoAtlas_corrected_{}_relabelled_to_match.gii'.format(row.subject_id, row.session_id, row.subject_id, row.session_id,  hemi)
    aims.write(tex, outf)
    
    if os.path.isfile(outf):
        print(row.subject_id, '{} DONE'.format(hemi))

('CC00150AN02', 'right DONE')
('CC00150AN02', 'left DONE')
('CC00554XX10', 'right DONE')
('CC00554XX10', 'left DONE')
('CC00389XX19', 'right DONE')
('CC00389XX19', 'left DONE')
('CC00186AN14', 'right DONE')
('CC00186AN14', 'left DONE')


In [1]:
import pandas as pd 
import numpy as np
from soma import aims
import os

In [2]:
## paleta vobec nefunguje lebo anatomist je pravdepodobne ten najhlupejsi software na visualizaciu, 
## skusim pre-labelovat jednu hemi na druhu aby som pouzila jednotnu schemu? 
sub = pd.read_csv('../../Scripts/data_processing/subj_for_visualisation.csv', names=['subject_id', 'session_id', 'template'])

In [13]:
for i, row in sub[:1].iterrows():
    h = 'R'
    hemi ='right'
    ## right
    mesh = aims.read('/neurospin/grip/external_databases/dHCP_CR_JD_2018/release3/dhcp_anat_pipeline/sub-{}/ses-{}/anat/sub-{}_ses-{}_T2w_{}white_bv_transformed.gii'.format(row.subject_id, row.session_id, row.subject_id, row.session_id,  h))
    vert, poly = np.array(mesh.vertex()) ,np.array(mesh.polygon())    
    iSegm = aims.read('../../DerivedData/subjects/sub-{}/ses-{}/sub-{}_ses-{}_neoAtlas_corrected_right.gii'.format(
                    row.subject_id, row.session_id, row.subject_id, row.session_id))
    segm = np.array(iSegm)[0].astype(int)
    
    
    tex = aims.TimeTexture('FLOAT')  
    for i in range(mesh.size()):
        tex[i].assign(np.zeros((len(vert),), dtype=int))
        t = np.asarray(tex[i])
        t[:] = segm[:]  
        
    outf = '../../Results/figures/example_subject_vis/CC00150AN/neoAtlas_{}_int.label.gii'.format( hemi)
    aims.write(tex, outf)  
    
    
    h = 'L'
    hemi ='left'
    ## right
    mesh = aims.read('/neurospin/grip/external_databases/dHCP_CR_JD_2018/release3/dhcp_anat_pipeline/sub-{}/ses-{}/anat/sub-{}_ses-{}_T2w_{}white_bv_transformed.gii'.format(row.subject_id, row.session_id, row.subject_id, row.session_id,  h))
    vert, poly = np.array(mesh.vertex()) ,np.array(mesh.polygon())    
    iSegm = aims.read('../../DerivedData/subjects/sub-{}/ses-{}/sub-{}_ses-{}_neoAtlas_corrected_left.gii'.format(
                    row.subject_id, row.session_id, row.subject_id, row.session_id))
    segm = np.array(iSegm)[0].astype(int)
    
    
    tex = aims.TimeTexture('FLOAT')  
    for i in range(mesh.size()):
        tex[i].assign(np.zeros((len(vert),), dtype=int))
        t = np.asarray(tex[i])
        t[:] = segm[:]  
        
    outf = '../../Results/figures/example_subject_vis/CC00150AN/neoAtlas_{}_int.label.gii'.format(  hemi)
    aims.write(tex, outf)  

In [14]:
segm

array([68, 68, 68, ...,  0,  0,  0])